In [17]:
import boto3
import os

def bucket_list():
    s3 = boto3.resource("s3", aws_access_key_id = key_id, aws_secret_access_key
 = secret_key)
    buckets = []
    for bucket in s3.buckets.all():
        #print(bucket.name)
        buckets.append(bucket.name)
    return buckets

def create_bucket(bucket_name, region):
    
    buckets = bucket_list()
    if(bucket_name in buckets):
        print("Bucket with this name already exists")
        return False
    try:
        if region is None:
            s3_client = boto3.client('s3', aws_access_key_id = key_id, 
                             aws_secret_access_key = secret_key)
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', aws_access_key_id = key_id, 
                                     aws_secret_access_key = secret_key,
                                     region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        print(e)
        return False
    
    print(response)
    return True

def destroy_bucket(bucket_name):
    buckets = bucket_list()
    if(bucket_name not in buckets):
        print("Bucket with this name does not exists")
        return False
    s3_client = boto3.client('s3', aws_access_key_id = key_id, 
                             aws_secret_access_key = secret_key)
    response = s3_client.delete_bucket(Bucket=bucket_name)
    print(response)
    return True


def create_key_pair(region = "us-east-1", keyName = "ec2-key-pair"):
    ec2 = boto3.client('ec2', aws_access_key_id = key_id, 
                             aws_secret_access_key = secret_key, 
                             region_name=region)
    key_pair = ec2.create_key_pair(KeyName = keyName)
    private_key = key_pair["KeyMaterial"]
    if (os.path.isfile("aws_ec2_key.pem")):
        os.remove("aws_ec2_key.pem")
    with os.fdopen(os.open("aws_ec2_key.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
    return True


def delete_key_pair(region = "us-east-1", keyName = "ec2-key-pair"):
    if (os.path.isfile("aws_ec2_key.pem")):
        os.remove("aws_ec2_key.pem")
    ec2 = boto3.client('ec2', aws_access_key_id = key_id, 
                       aws_secret_access_key = secret_key,
                       region_name=region)
    response = ec2.delete_key_pair(KeyName=keyName)
    print(response)
    return True


from botocore.exceptions import ClientError


with open('cred') as f:
    lines = f.readlines()
key_id = (lines[0])[:-1]
secret_key = lines[1]

In [21]:
def get_running_instances(region):
    ec2_client = boto3.client("ec2", aws_access_key_id = key_id, 
                             aws_secret_access_key = secret_key, 
                             region_name=region)
    reservations = ec2_client.describe_instances(Filters=[
    {
        "Name": "instance-state-name",
        "Values": ["running"],
    },
    {
    "Name": "instance-type",
    "Values": ["t2.micro"]
    }]).get("Reservations")
    for reservation in reservations:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            instance_type = instance["InstanceType"]
            public_ip = instance["PublicIpAddress"]
            private_ip = instance["PrivateIpAddress"]
            print(f"{instance_id}, {instance_type}, {public_ip},{private_ip}")

In [22]:
get_running_instances(region = "us-east-1")

i-08f5a8c681879b836, t2.micro, 54.158.88.10,172.31.94.147


In [11]:
bucket_list()

['truezhesha']

In [49]:
delete_key_pair(region = "us-east-1", keyName = "ec2-key-pair")

{'ResponseMetadata': {'RequestId': '9e2c6e08-bf31-46e7-b330-cbefb5c19547', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9e2c6e08-bf31-46e7-b330-cbefb5c19547', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '276', 'date': 'Fri, 26 May 2023 12:57:46 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


True

In [51]:
create_key_pair(region = "us-east-1", keyName = "ec2-key-pair")

True

In [22]:
def upload(file_path, bucket_name, path):
    try:
        s3_client = boto3.client('s3', aws_access_key_id = key_id, 
                                 aws_secret_access_key = secret_key)
        response = s3_client.upload_file(Filename=file_path, Bucket=bucket_name, Key=path)
        print(response)
    except ClientError as e:
        print(e)
        return False
    
    return True

In [23]:
upload('cur_df.csv', 'truezhesha', 'cur_df.csv')

None


True

In [61]:
def create_instance(region, keyName):
    try:
        ec2_client = boto3.client('ec2', aws_access_key_id = key_id, 
                                  aws_secret_access_key = secret_key,
                                  region_name=region)
        instances = ec2_client.run_instances(ImageId="ami-0bef6cc322bfff646",
                                             MinCount=1,
                                             MaxCount=1,
                                             InstanceType="t2.micro",
                                             KeyName=keyName)
        
    except ClientError as e:
        print(e)
        return False
    return True

In [62]:
create_instance(region = "us-east-1", keyName = "ec2-key-pair")

In [13]:
def get_public_ip(instance_id, region):
    ec2_client = boto3.client("ec2", aws_access_key_id = key_id, 
                                  aws_secret_access_key = secret_key,
                                  region_name=region)
    try:
        reservations = ec2_client.describe_instances(InstanceIds=[instance_id]).get("Reservations")

        for reservation in reservations:
            for instance in reservation['Instances']:
                print(instance.get("PublicIpAddress"))
    except ClientError as e:
        print(e)
        return False            
    return True

In [14]:

get_public_ip(region = "us-east-1", instance_id ='i-08f5a8c681879b836')

54.163.183.200


True

In [9]:
def stop_instance(region, instance_id):
    try:
        ec2_client = boto3.client("ec2", aws_access_key_id = key_id, 
                                  aws_secret_access_key = secret_key,
                                  region_name=region)
        response = ec2_client.stop_instances(InstanceIds=[instance_id])
        print(response)
    except ClientError as e:
        print(e)
        return False
    return True

In [10]:

stop_instance(region = "us-east-1", instance_id ='i-0a47be85a9de8c3ba')

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-0a47be85a9de8c3ba', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '36157755-ba7d-41fb-ad3a-93587cd7ad8b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '36157755-ba7d-41fb-ad3a-93587cd7ad8b', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Fri, 26 May 2023 15:00:23 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


True

In [12]:

stop_instance(region = "us-east-2", instance_id ='i-0a47be85a9de8c3ba')

An error occurred (InvalidInstanceID.NotFound) when calling the StopInstances operation: The instance ID 'i-0a47be85a9de8c3ba' does not exist


False

In [25]:
def start_instance(region, instance_id):
    try:
        ec2_client = boto3.client("ec2", aws_access_key_id = key_id, 
                                  aws_secret_access_key = secret_key,
                                  region_name=region)
        response = ec2_client.start_instances(InstanceIds=[instance_id])
        print(response)
    except ClientError as e:
        print(e)
        return False
    return True

In [26]:

start_instance(region = "us-east-1", instance_id ='i-0a47be85a9de8c3ba')

{'StartingInstances': [{'CurrentState': {'Code': 0, 'Name': 'pending'}, 'InstanceId': 'i-0a47be85a9de8c3ba', 'PreviousState': {'Code': 80, 'Name': 'stopped'}}], 'ResponseMetadata': {'RequestId': '2158609b-f503-44c8-9cfe-17be6179b276', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2158609b-f503-44c8-9cfe-17be6179b276', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Fri, 26 May 2023 16:06:00 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


True

In [27]:
def terminate_instance(region, instance_id):
    try:
        ec2_client = boto3.client("ec2", aws_access_key_id = key_id, 
                                  aws_secret_access_key = secret_key,
                                  region_name=region)
        response = ec2_client.terminate_instances(InstanceIds=[instance_id])
        print(response)
    except ClientError as e:
        print(e)
        return False
    return True

In [28]:
terminate_instance(region = "us-east-1", instance_id ='i-0a47be85a9de8c3ba')


{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0a47be85a9de8c3ba', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'a1440f8a-22a4-4594-85d9-b28e01336bb7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a1440f8a-22a4-4594-85d9-b28e01336bb7', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'accept-encoding', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'date': 'Fri, 26 May 2023 16:06:59 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


True

In [40]:

create_bucket("truezhesha", "us-east-1")

Bucket with this name already exists


False

In [42]:
create_bucket("truezhesha2", 'eu-west-1')

True

In [54]:
destroy_bucket("bucket")

Bucket with this name does not exists


False

In [55]:
destroy_bucket("truezhesha2")

{'ResponseMetadata': {'RequestId': '7E6T3MDXCC20FET6', 'HostId': 'iHADZpE7+tf0gwXfVgba+C7DNHdEeeLwtYZClMqGBEvB+qsAtzzLiPpnuF9qiO/ktzGoPljtTAk=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'iHADZpE7+tf0gwXfVgba+C7DNHdEeeLwtYZClMqGBEvB+qsAtzzLiPpnuF9qiO/ktzGoPljtTAk=', 'x-amz-request-id': '7E6T3MDXCC20FET6', 'date': 'Fri, 26 May 2023 13:03:37 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 1}}


True

In [25]:
def get_object(bucket_name, key):

    try:
        s3_client = boto3.client('s3')
        obj = s3_client.get_object(
            Bucket = bucket_name,
            Key = key
        )
    except ClientError as e:
        print(e)
        return False
    return obj


In [26]:
obj = get_object("truezhesha", "cur_df.csv")
df = pandas.read_csv(obj['Body'])
df

,exchangedate,r030,cc,txt,enname,rate,units,rate_per_unit,group,calcdate,date
0,01.01.2021,840,USD,Долар США,US Dollar,28.2746,1,28.2746,1,30.12.2020,2021-01-01
1,01.01.2021,978,EUR,Євро,Euro,34.7396,1,34.7396,1,30.12.2020,2021-01-01
2,02.01.2021,978,EUR,Євро,Euro,34.7396,1,34.7396,1,30.12.2020,2021-01-02
3,02.01.2021,840,USD,Долар США,US Dollar,28.2746,1,28.2746,1,30.12.2020,2021-01-02
4,03.01.2021,840,USD,Долар США,US Dollar,28.2746,1,28.2746,1,30.12.2020,2021-01-03
...,...,...,...,...,...,...,...,...,...,...,...
725,29.12.2021,978,EUR,Євро,Euro,30.8401,1,30.8401,1,28.12.2021,2021-12-29
726,30.12.2021,840,USD,Долар США,US Dollar,27.2077,1,27.2077,1,29.12.2021,2021-12-30
727,30.12.2021,978,EUR,Євро,Euro,30.7760,1,30.7760,1,29.12.2021,2021-12-30
728,31.12.2021,978,EUR,Євро,Euro,30.9226,1,30.9226,1,30.12.2021,2021-12-31


In [27]:
obj = get_object("truezhesha", "cur12_df.csv")

An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
